# Binary Trees
Formally, a binary tree is either empty, or a *root* node $r$ together with a left binary tree and a right binary tree. The subtrees themselves are binary trees.

## Tips
- **Recursive algorithms** are well-suited to problems on trees. Remember to include space implicitly allocated on the **function call stack** when doing space complexity analysis. 
- Some tree problems have a simple brute-force solution that allocates $O(n)$ additional space, but a more sophisticated solution that uses $O(1)$.
- Consider **left- and right-skewed trees** when doing complexity analysis. Note that $O(h)$ complexity, where $h$ is the tree height, translates into $O(\log n)$ complexity for balanced trees, but $O(n)$ comlexity for skewed trees.
- If each node has a **parent field**, use it to make your code simpler, and to reduce time and space complexity.
- It's easy to make the **mistake** of treating a node that has a **single child** as a leaf.

In [2]:
from typing import Deque, Iterator, List, Optional
from collections import deque, namedtuple
from dataclasses import dataclass

from utils import run_tests
import trees
from trees import BinaryTreeNode

In [3]:
tree = trees.make_tree_example()

### 9.1: Test if a Binary Tree is Height Balanced

In [4]:
def is_height_balanced(root: BinaryTreeNode) -> bool:

    def height_helper(node: BinaryTreeNode) -> int:
        if not node:
            return 0
        else:
            return max([height_helper(node.left), height_helper(node.right)]) + 1

    # base condition
    if root is None:
        return True
    
    # get tree height of left and right subtree
    left_height, right_height = height_helper(root.left), height_helper(root.right)
    if abs(left_height - right_height) <= 1 and is_height_balanced(root.left) and is_height_balanced(root.right): 
        return True

    return False
    

In [5]:
is_height_balanced(tree)

False

In [6]:
tree_balanced = BinaryTreeNode(4, left=BinaryTreeNode(2), right=BinaryTreeNode(1))
is_height_balanced(tree_balanced)

True

Time complexity is $O(n^2)$ because calling height function on each node while recursively checking if height balanced

In [7]:
def is_height_balanced(root: BinaryTreeNode) -> bool:
    BalancedStatusWithHeight = namedtuple('BalancedStatusWithHeight', ('balanced', 'height'))

    def check_balance(tree: BinaryTreeNode):
        if not tree:
            return BalancedStatusWithHeight(balanced=True, height=-1)

        left_result = check_balance(tree.left)
        if not left_result.balanced:
            return left_result
        
        right_result = check_balance(tree.right)
        if not right_result.balanced:
            return right_result
        
        is_balanced = abs(left_result.height - right_result.height) <= 1
        height = max([left_result.height, right_result.height]) + 1
        return BalancedStatusWithHeight(is_balanced, height)

    return check_balance(root).balanced

Time complxity is $O(n)$ (same as post-order traversal) and space complexity is $O(h)$

In [8]:
is_height_balanced(tree)

False

In [9]:
tree_balanced = BinaryTreeNode(4, left=BinaryTreeNode(2), right=BinaryTreeNode(1))
is_height_balanced(tree_balanced)

True

### Check if a Binary Tree is Complete

In [10]:
def is_complete(tree: BinaryTreeNode) -> bool:
    ''' 
    level order
    '''
    if not tree:
        return True 

    queue = deque()
    queue.append(tree)

    # flag to mark the end of full nodes
    flag = False 

    # loop til queue is empty
    while queue:

        node = queue.popleft()

        # if have encountered a non-full node before and the current node
        # is not a leaf, cannot be complete
        if flag and (node.left or node.right):
            return False 

        # if left child is empty and right chiled exists,
        # cannot be complete
        if not node.left and node.right:
            return False 
        
        # if left child exists
        if node.left:
            queue.append(node.left)
        # current node is non-full
        else:
            flag = True
        
         # if right child exists
        if node.right:
            queue.append(node.right)
        # current node is non-full
        else:
            flag = True    

    return True

print(is_complete(tree))
tree_complete_eg = BinaryTreeNode(5)
tree_complete_eg.left = BinaryTreeNode(5)
tree_complete_eg.right = BinaryTreeNode(10)
print(is_complete(tree_complete_eg))   


False
True


$O(n)$ time and space complexity

In [11]:
def is_complete(tree: BinaryTreeNode):
    ''' 
    if binary tree is represented as an array, 
    all array postiions should be filled if complete
    '''
    def size(tree: BinaryTreeNode) -> int:
        if not tree:
            return 0
        return 1 + size(tree.left) + size(tree.right)

    def inorder(tree: BinaryTreeNode, A: List[int], i: int):
        if not tree or i >= len(A):
            return 
        
        inorder(tree.left, A, 2*i + 1)
        A[i] = True
        inorder(tree.right, A, 2*i + 2)

    
    n = size(tree)
    A = [False] * n

    # fill in values of A
    inorder(tree, A, 0)

    # see if all postitions are filled 
    for e in A:
        if not e:
            return False 
    
    return True

    
print(is_complete(tree))
print(is_complete(tree_complete_eg))   

False
True


$O(n)$ time and space complexity

In [12]:
def is_complete(tree: BinaryTreeNode):
    ''' 
    if binary tree is represented as an array, 
    all array postiions should be filled if complete
    '''
    def size(tree: BinaryTreeNode) -> int:
        if not tree:
            return 0
        return 1 + size(tree.left) + size(tree.right)
    
    def check_complete(tree: BinaryTreeNode, i: int, n: int) -> bool:

        if not tree:
            return True 
        
        if (tree.left and 2*i + 1 >= n) or not check_complete(tree.left, 2*i + 1, n):
            return False 

        if (tree.right and 2*i + 2 >= n) or not check_complete(tree.right, 2*i + 2, n):
            return False 

        return True 

    n = size(tree)
    return check_complete(tree, 0, n)

print(is_complete(tree))
print(is_complete(tree_complete_eg))   

False
True


$O(n)$ time and $O(h)$ space complexity

#### Variant: Return the size of the largest subtree that is complete

### 9.2: Test if a Binary Tree is Symmetric

In [13]:
def is_symmetric(tree: BinaryTreeNode) -> bool:

    def is_symmetric_helper(sub_tree0: BinaryTreeNode, sub_tree1: BinaryTreeNode) -> bool:
        if not sub_tree0 and not sub_tree1:
            return True
        elif sub_tree0 and sub_tree1:
            return (sub_tree0.data == sub_tree1.data and
                    is_symmetric_helper(sub_tree0.left, sub_tree1.right) and
                    is_symmetric_helper(sub_tree0.right, sub_tree1.left)
                    )
        # one subtree is empty and the other is not
        else:
            return False
    return not tree or is_symmetric_helper(tree.left, tree.right)

Time and space complexity is $O(n)$ and $O(h)$ respectively

In [14]:
is_symmetric(tree)

False

In [15]:
is_symmetric(BinaryTreeNode(5, 
                left=BinaryTreeNode(10, 
                    left=None, 
                    right=BinaryTreeNode(100)
                ),
                right=BinaryTreeNode(10,
                    left=BinaryTreeNode(100),
                    right=None
                )
            )
        )
    

True

### 9.3: Find the Least Common Ancestor of Two Nodes

### 9.4: Find the Least Common Ancestor of Two Nodes with Parent Nodes

In [16]:
class BinaryTreeNodeParent:

    def __init__(self, data=None, left=None, right=None, parent=None) -> None:
        self.data = data
        self.left = left 
        self.right = right 
        self.parent = parent


In [17]:
root = BinaryTreeNodeParent(data=100)
ln = BinaryTreeNodeParent(parent=root, data=10)
rn = BinaryTreeNodeParent(parent=root, data=15)
root.left, root.right = ln, rn
lln = BinaryTreeNodeParent(data=4, parent=ln)



In [18]:
# walk up tree until get to lca
def lca_parent(node0: BinaryTreeNodeParent, node1: BinaryTreeNodeParent) -> BinaryTreeNodeParent:

    # calculate depth of each node; if there is a difference, need to walk up longer one
    def get_depth(node: BinaryTreeNodeParent) -> int:
        depth = 0
        while node.parent:
            depth += 1
            node = node.parent
        return depth 
    
    depth_node0, depth_node1 = get_depth(node0), get_depth(node1)
    # make node0 longer node to simplify code
    if depth_node1 > depth_node0:
        node0, node1 = node1, node0 
    
    depth_diff = abs(depth_node0 - depth_node1)
    while depth_diff:
        node0 = node0.parent
        depth_diff -= 1
    
    # walk up tree til nodes same
    while node0 is not node1:
        node0, node1 = node0.parent, node1.parent
    
    return node0

The time and space complexity are that of computing the depth, namely $O(h)$ and $O(1)$, respectively

In [19]:
lca_parent(lln, rn).data

100

### 9.5: Sum the Root-to-Leaf Paths in a Binary Tree

In [20]:
def sum_root_to_leaf(tree: BinaryTreeNode) -> int:

    def helper(tree: BinaryTreeNode, partial_sum: int=0):
        if not tree:
            return 0
        
        partial_sum = partial_sum * 2 + tree.data 

        if not tree.left and not tree.right:  # leaf
            return partial_sum
        else:   # non-leaf
            return helper(tree.left, partial_sum) + helper(tree.right, partial_sum)
    
    return helper(tree)

In [21]:
binary_tree = BinaryTreeNode(1,
                left=BinaryTreeNode(0,
                    left=BinaryTreeNode(0,
                        left=BinaryTreeNode(0),
                        right=BinaryTreeNode(1)
                        ),
                    right=BinaryTreeNode(1,
                        left=None,
                        right=BinaryTreeNode(1,
                            left=BinaryTreeNode(0),
                            right=None
                            )
                        )
                    ),
                right=BinaryTreeNode(1,
                    left=BinaryTreeNode(0,
                        left=None,
                        right=BinaryTreeNode(0,
                            left=BinaryTreeNode(1,
                                left=None,
                                right=BinaryTreeNode(1)
                                ),
                            right=BinaryTreeNode(0)
                            )
                        ),
                    right=BinaryTreeNode(0,
                        left=None,
                        right=BinaryTreeNode(0))
                    )
                )
trees.traversal_levelorder(binary_tree)

1 0 1 0 1 0 0 0 1 1 0 0 0 1 0 1 

In [22]:
sum_root_to_leaf(binary_tree)

126

Time and space complexity is $O(n)$ and $O(h)$ respectively

### 9.6: Find a Root to Leaf Path with Specified Sum

In [23]:
def has_path_sum(tree: BinaryTreeNode, remaining_weight: int) -> bool:
    if not tree:
        return None

    if not tree.left and not tree.right:   # leaf
        return remaining_weight == tree.data

    # non-leaf 
    return has_path_sum(tree.left, remaining_weight - tree.data) or has_path_sum(tree.right, remaining_weight - tree.data)

Time and space complexity is $O(n)$ and $O(h)$ respectively

In [24]:
has_path_sum(tree, 591)

True

### 9.7: Implement an Inorder Traversal without Recursion

In [25]:
def inorder_traversal_no_recursion(tree: BinaryTreeNode) -> List[int]:
    result = []

    if not tree:
        return result

    in_process = [(tree, False)]   # used as stack
    while in_process:
        node, left_subtree_traversed = in_process.pop()
        if node:
            if left_subtree_traversed:
                result.append(node.data)
            else:
                # process left subtree, print root, process right subtree
                in_process.append((node.right, False))
                in_process.append((node, True))
                in_process.append((node.left, False))
    
    return result

Time and space complexity is $O(n)$ and $O(h)$ respectively

In [26]:
inorder_traversal_no_recursion(tree)

[28, 271, 0, 6, 561, 17, 3, 314, 2, 401, 641, 1, 257, 7, 272, 29]

#### Variant: Postorder Traversal without Recursion

In [27]:
def postorder_traversal_no_recursion(tree: BinaryTreeNode) -> List[int]:
    result = []

    if not tree:
        return result

    in_process = [(tree, False)]    # used as stack
    while in_process:
        node, left_subtree_traversed = in_process.pop()
        if node:
            if left_subtree_traversed:
                result.append(node.data)
            else:
                # process left subtree, right subtree, print root
                in_process.append((node, True))
                in_process.append((node.right, False))
                in_process.append((node.left, False))
    
    return result

In [28]:
postorder_traversal_no_recursion(tree)

[28, 0, 271, 17, 3, 561, 6, 641, 401, 257, 1, 2, 29, 272, 7, 314]

#### Variant: Preorder Traversal without Recursion

In [29]:
def preorder_traversal_no_recursion(tree: BinaryTreeNode) -> List[int]:
    result = []

    if not tree:
        return result

    in_process = [(tree, False)]    # used as stack
    while in_process:
        node, left_subtree_traversed = in_process.pop()
        if node:
            if left_subtree_traversed:
                result.append(node.data)
            else:
                # print node, process left subtree, right subtree
                in_process.append((node.right, False))
                in_process.append((node.left, False))
                in_process.append((node, True))
    
    return result

In [30]:
preorder_traversal_no_recursion(tree)

[314, 6, 271, 28, 0, 561, 3, 17, 7, 2, 1, 401, 641, 257, 272, 29]

### *9.8: Find Kth Node in Binary Tree
Kth node in an inorder traversal   
Assume you have size of tree rooted at particular node

In [31]:
def add_size(tree: BinaryTreeNode) -> None:
    if not tree:
        return

    tree.size = size(tree)
    add_size(tree.left)
    add_size(tree.right)

add_size(tree)

NameError: name 'size' is not defined

In [ ]:
# if k is greater than the number of nodes in the left subtree, k cannot lie in the left subtree
# if the left subtree contains L nodes, then kth node in the original tree is the (k-L)th node when skipping the left subtree
def find_kth_node(tree: BinaryTreeNode, k: int) -> Optional[BinaryTreeNode]:
    
    while tree:
        left_size = tree.left.size if tree.left else 0
        if left_size + 1 < k:    # kth node in right sub tree
            k -= left_size + 1   # + 1 for root node 
            tree = tree.right 
        elif left_size == k - 1: # kth node
            return tree
        else:                    # kth node in left subtree
            tree = tree.left
    
    return None  # kth unreachable

Time complexity is $O(h)$

In [ ]:
trees.traversal_inorder(tree)
print()
print(find_kth_node(tree, 3).data)
print(find_kth_node(tree, 10).data)

28 271 0 6 561 17 3 314 2 401 641 1 257 7 272 29 
0
401


### 9.9: Compute Successor

### 9.13: Compute Leaves of Binary Tree

In [ ]:
def compute_leaves(tree: BinaryTreeNode) -> List[int]:
    def leaves_helper(tree: BinaryTreeNode) -> None:
        if tree:
            leaves_helper(tree.left)
            leaves_helper(tree.right)
            if not tree.left and not tree.right:
                result.append(tree.data)
    
    result = []

    if not tree:
        return result 

    leaves_helper(tree)
    
    return result
    
compute_leaves(tree) 

[28, 0, 17, 641, 257, 29]

In [ ]:
def compute_leaves(tree: BinaryTreeNode) -> List[int]:
    if not tree:
        return []
    if not tree.left and not tree.right:
        return [tree.data]
    return compute_leaves(tree.left) + compute_leaves(tree.right)

compute_leaves(tree)

[28, 0, 17, 641, 257, 29]

$O(n)$ time complexity 